In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        G = tsp.get_graph()
        G = imp_nx.read(G)
        m = gen_path_atsp.createGenModel(G,'min','metric')
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1

    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/kroB150.tsp",
             "TSPTestInstances/bayg29.tsp",
             "TSPTestInstances/bier127.tsp",
             ]

    """TSPTestInstances/d1655.tsp",
             "TSPTestInstances/pr1002.tsp"""
    
    runBenchmark(paths)

Using license file /opt/gurobi/gurobi.lic
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 1500.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 22651 rows, 22650 columns and 111753 nonzeros
Model fingerprint: 0x43f0d7cc
Variable types: 0 continuous, 22650 integer (22500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [8e+00, 4e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Presolve removed 150 rows and 150 columns
Presolve time: 0.20s
Presolved: 22501 rows, 22500 columns, 111156 nonzeros
Variable types: 0 continuous, 22500 integer (22351 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simp

  9177  5763     cutoff   94      26769.0000 25322.0000  5.41%  28.1  286s
  9698  6234 25477.9457   53  426 26769.0000 25322.0000  5.41%  28.7  291s
 10272  6817 26095.3408  167  285 26769.0000 25322.0000  5.41%  29.3  296s
 10896  7411 25683.4386   75  328 26769.0000 25322.0000  5.41%  30.0  301s
 11642  8029 25502.2171   69  325 26769.0000 25324.3778  5.40%  30.6  308s
H11697  7612                    26617.000000 25324.3778  4.86%  30.7  308s

Cutting planes:
  Learned: 40
  Gomory: 62
  Implied bound: 17
  Projected implied bound: 37
  MIR: 70
  Flow cover: 256
  Inf proof: 9
  Zero half: 56
  RLT: 17
  Relax-and-lift: 48

Explored 12010 nodes (373674 simplex iterations) in 308.23 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 26617 26769 26949 ... 27726

Optimal solution found (tolerance 5.00e-02)
Best objective 2.661700000000e+04, best bound 2.532500000000e+04, gap 4.8540%
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf 

     0     0 112217.361    0  179          - 112217.361      -     -    3s
     0     0 112217.693    0  188          - 112217.693      -     -    3s
     0     0 112436.667    0  191          - 112436.667      -     -    3s
     0     0 112436.667    0  192          - 112436.667      -     -    3s
     0     0 112436.667    0  210          - 112436.667      -     -    4s
     0     0 112436.667    0  206          - 112436.667      -     -    4s
     0     0 112437.396    0  280          - 112437.396      -     -    4s
     0     0 112467.542    0  289          - 112467.542      -     -    4s
     0     0 112467.542    0  289          - 112467.542      -     -    4s
     0     0 112488.875    0  300          - 112488.875      -     -    4s
     0     0 112488.875    0  299          - 112488.875      -     -    5s
     0     0 112488.875    0  297          - 112488.875      -     -    5s
     0     0 112725.857    0  292          - 112725.857      -     -    5s
     0     0 112725.857  